# Example of Langchain Integration with an Ollama model (LLama2:7b quantified) 

In [1]:
from langchain_community.llms import Ollama

# Load the model
ollama = Ollama(base_url="http://localhost:11434", model="llama2:7b-chat-q4_0")

# print(ollama.invoke("who is the president of the united states?"))

## Case : RAG from wikipedia page to retrieve contextual info

In [2]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

# Load the wikipedia page
loader = WebBaseLoader("https://www.lebigdata.fr/comprendre-les-modeles-de-vision-et-de-langage-un-regard-sur-le-vlm")
docs = loader.load()

# Split the page into chunks    
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(docs)

# Embed the chunks
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}
)
# Create a vector store
vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=embedding
)

# Create a retrieval chain
qacahin = RetrievalQA.from_chain_type(
    llm=ollama, retriever=vectorstore.as_retriever()
)

# Query the chain
query = "Résume le contenu de la page en moins de 500 mots."

print(qacahin({'query': query}))

USER_AGENT environment variable not set, consider setting it to identify your requests.
/tmp/ipykernel_61520/2349481596.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding = HuggingFaceEmbeddings(
/tmp/ipykernel_61520/2349481596.py:34: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  print(qacahin({'query': query}))


{'query': 'Résume le contenu de la page en moins de 500 mots.', 'result': 'The page is a newsletter for an organization focused on artificial intelligence and virtual reality. The newsletter includes several articles and announcements, including:\n\n1. A headline about a company called "Chasseurs de bug" that received $12 million from Google in 2024.\n2. An article about a fusion blanket that could potentially produce energy like the sun on Earth if a test is successful.\n3. A news item about Elon Musk accusing Ukraine of causing a software problem, with Mars as the suspected location of the problem.\n4. A call to action to subscribe to the newsletter and receive updates on artificial intelligence and virtual reality trends.\n\nOverall, the page is focused on promoting the organization\'s interest in artificial intelligence and virtual reality, and providing news and updates on these topics.'}


## Temps d'inférence = 5min (très long)